In [1]:
import pandas as pd

In [8]:
df=pd.read_csv('/content/drive/MyDrive/final_project/review_data.csv')

In [9]:
df

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,love well made sturdy comfortable love itvery ...
1,Home_and_Kitchen_5,5.0,CG,love great upgrade original ive mine couple year
2,Home_and_Kitchen_5,5.0,CG,pillow saved back love look feel pillow
3,Home_and_Kitchen_5,1.0,CG,missing information use great product price
4,Home_and_Kitchen_5,5.0,CG,nice set good quality set two month
...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,read review saying bra ran small ordered two b...
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,wasnt sure exactly would little large small si...
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,wear hood wear hood wear jacket without hood s...
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,liked nothing dress reason gave star ordered s...


# tfidf- lstm

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Reshape
from sklearn.metrics import classification_report,accuracy_score
from sklearn.preprocessing import LabelEncoder

# Convert text to features
# TF-IDF Vectorizer
tfidf = TfidfVectorizer(max_features=5000)  # Adjust max_features as needed
X = tfidf.fit_transform(df["text_"]).toarray()

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Transform the labels
y = label_encoder.fit_transform(df["label"])

# Apply PCA for dimensionality reduction
pca = PCA(n_components=100)  # Reduce to 100 components
X_reduced = pca.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

# LSTM Model
lstm_model = Sequential([
    Reshape((X_train.shape[1], 1), input_shape=(X_train.shape[1],)),
    LSTM(128, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

# Compile the model
lstm_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
lstm_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 130s 126ms/step - accuracy: 0.5026 - loss: 0.6934 - val_accuracy: 0.4966 - val_loss: 0.6934
Epoch 2/10
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 154s 138ms/step - accuracy: 0.4955 - loss: 0.6934 - val_accuracy: 0.5294 - val_loss: 0.6930
Epoch 3/10
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 129s 125ms/step - accuracy: 0.5064 - loss: 0.6931 - val_accuracy: 0.4955 - val_loss: 0.6931
Epoch 4/10
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 146s 129ms/step - accuracy: 0.5058 - loss: 0.6929 - val_accuracy: 0.4870 - val_loss: 0.6984
Epoch 5/10
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 173s 161ms/step - accuracy: 0.5160 - loss: 0.6918 - val_accuracy: 0.5343 - val_loss: 0.6917
Epoch 6/10
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 167s 126ms/step - accuracy: 0.5243 - loss: 0.6918 - val_accuracy: 0.4919 - val_loss: 0.6923
Epoch 7/10
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 130s 128ms/step - accuracy: 0.5096 - loss: 0.6928 - val_accuracy: 0.5264 - val_loss: 0.6911
Epoch 8/10
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 138s 124ms/step - ac

In [ ]:
# Generate probabilities
y_pred_probs = lstm_model.predict(X_test)

# Convert probabilities to binary predictions (threshold = 0.5)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

# Accuracy Score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

target_names = label_encoder.classes_

# Classification Report
report = classification_report(y_test, y_pred, target_names=target_names)
print("Classification Report:")
print(report)


253/253 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step
Accuracy: 0.5174
Classification Report:
              precision    recall  f1-score   support

          CG       0.64      0.06      0.12      4016
          OR       0.51      0.97      0.67      4071

    accuracy                           0.52      8087
   macro avg       0.58      0.51      0.39      8087
weighted avg       0.58      0.52      0.39      8087



# countvectorizer

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Reshape
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Convert text to features
# CountVectorizer
count_vectorizer = CountVectorizer(max_features=5000)  # Adjust max_features as needed
X = count_vectorizer.fit_transform(df["text_"]).toarray()

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Transform the labels
y = label_encoder.fit_transform(df["label"])

# Apply PCA for dimensionality reduction
pca = PCA(n_components=100)  # Reduce to 100 components
X_reduced = pca.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

# LSTM Model
lstm_model = Sequential([
    Reshape((X_train.shape[1], 1), input_shape=(X_train.shape[1],)),
    LSTM(128, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

# Compile the model
lstm_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
lstm_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Generate probabilities
y_pred_probs = lstm_model.predict(X_test)

# Convert probabilities to binary predictions (threshold = 0.5)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

# Accuracy Score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Target Names
target_names = label_encoder.classes_

# Classification Report
report = classification_report(y_test, y_pred, target_names=target_names)
print("Classification Report:")
print(report)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1011/1011 ━━━━━━━━━━━━━━━━━━━━ 131s 127ms/step - accuracy: 0.5042 - loss: 0.6931 - val_accuracy: 0.5185 - val_loss: 0.6924
Epoch 2/10
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 152s 137ms/step - accuracy: 0.5158 - loss: 0.6925 - val_accuracy: 0.4965 - val_loss: 0.6925
Epoch 3/10
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 132s 127ms/step - accuracy: 0.5126 - loss: 0.6918 - val_accuracy: 0.5409 - val_loss: 0.6901
Epoch 4/10
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 128s 127ms/step - accuracy: 0.5176 - loss: 0.6907 - val_accuracy: 0.5509 - val_loss: 0.6872
Epoch 5/10
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 131s 130ms/step - accuracy: 0.5347 - loss: 0.6890 - val_accuracy: 0.5517 - val_loss: 0.6866
Epoch 6/10
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 138s 126ms/step - accuracy: 0.5405 - loss: 0.6876 - val_accuracy: 0.5576 - val_loss: 0.6786
Epoch 7/10
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 145s 128ms/step - accuracy: 0.5639 - loss: 0.6749 - val_accuracy: 0.5866 - val_loss: 0.6603
Epoch 8/10
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 138s 125ms/step - accuracy: 0.5